In [1]:
import numpy as np
caffe_root = '../../'
import sys
sys.path.insert(0, caffe_root+'python')
import caffe

#net = caffe.Net('../nin_sc_new/deploy.prototxt','../nin_sc_new/cifar10_nin_supercategory_iter_60000.caffemodel', caffe.TEST)
#net = caffe.Net('../super_category_nin/deploy.prototxt','../super_category_nin/cifar10_nin_supercategory_iter_60000.caffemodel', caffe.TEST)
net = caffe.Net('./deploy.prototxt','../base.caffemodel', caffe.TEST)
caffe.set_mode_gpu()

In [22]:
def predict(net, inputs, oversample=True):
    """
    Predict classification probabilities of inputs.
    Parameters
    ----------
    inputs : iterable of (H x W x K) input ndarrays.
    oversample : boolean
        average predictions across center, corners, and mirrors
        when True (default). Center-only prediction when False.
    Returns
    -------
    predictions: (N x C) ndarray of class probabilities for N images and C
        classes.
    """
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    
    if oversample:
        # Generate center, corner, and mirrored crops.
        input_ = caffe.io.oversample(inputs, [26,26])
    else:
        # Take center crop.
        center = np.array([32,32]) / 2.0
        crop = np.tile(center, (1, 2))[0] + np.concatenate([
            np.array([-26,-26]) / 2.0,
            np.array([26,26]) / 2.0
        ])
        input_ = np.array(inputs)[:, crop[0]:crop[2], crop[1]:crop[3], :]

    # Classify
    caffe_in = np.zeros(np.array(input_.shape)[[0, 3, 1, 2]],
                        dtype=np.float32)
    for ix, in_ in enumerate(input_):
        caffe_in[ix] = transformer.preprocess(net.inputs[0], in_)
    out = net.forward_all(**{net.inputs[0]: caffe_in})
    predictions = out[net.outputs[0]]

    # For oversampling, average predictions across crops.
    if oversample:
        predictions = predictions.reshape((len(predictions) / 10, 10, -1))
        predictions = predictions.mean(1)

    return predictions

In [3]:
def makeLabelKeyDict() :
    dic = {}
    for i in range(0,10):
        dic[i] = []
    
    import lmdb
    env = lmdb.open('/home/nine/datasets/cifar10/cifar_test_lmdb')
    with env.begin() as txn:
        cursor = txn.cursor()
        for key,value in cursor : 
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(value)
            dic[datum.label].append(key)
    return dic

label_key = makeLabelKeyDict()

In [4]:
def getImage(key):
    import lmdb
    
    imgs = [];
    
    img_preprocessed = np.zeros([32,32,3],np.float)
    env = lmdb.open('/home/nine/datasets/cifar10/cifar_test_lmdb')
    with env.begin() as txn:
        cursor = txn.cursor()
        value = cursor.get(key)
        datum = caffe.proto.caffe_pb2.Datum()
        datum.ParseFromString(value)
        
        label = datum.label
        img_preprocessed[:,:,0] = np.array(datum.float_data)[0:1024].reshape(32,32)
        img_preprocessed[:,:,1] = np.array(datum.float_data)[1024:2048].reshape(32,32)
        img_preprocessed[:,:,2] = np.array(datum.float_data)[2048:].reshape(32,32)
        
    return (label,img_preprocessed)

In [5]:
labels = open('/home/nine/datasets/cifar10/label.txt').readlines()
labels = [label.strip() for label in labels]

In [23]:
cnt = 0
correct = 0
for i in range(0,10) :
    inner_cnt = 0
    inner_correct = 0
    for key in label_key[i]:
        label,img = getImage(key)
        prediction = predict(net,[img],oversample=False)
        if prediction.argmax() == label :
            correct = correct + 1
            inner_correct = inner_correct + 1
        cnt = cnt + 1
        inner_cnt = inner_cnt + 1
    print inner_correct, '/', inner_cnt
print correct,'/', cnt

931 / 1000
930 / 1000
752 / 1000
734 / 1000
880 / 1000
839 / 1000
918 / 1000
881 / 1000
927 / 1000
905 / 1000
8697 / 10000
